# MyMethods.jl -- Examples

This notebook illustrates basic functionalities of the MyMethods.jl package.

### Installation and Removal of MyMethods.jl

In [2]:
# Install the package from GitHub (first time only) 
#using Pkg; 
#Pkg.add(url="https://github.com/thomaswiemann/MyMethods.jl");

# Uninstall 
#Pkg.rm("MyMethods")

   Updating git-repo `https://github.com/thomaswiemann/MyMethods.jl`


   Updating registry at `C:\Users\thomas\.julia\registries\General`


LoadError: package `MyMethods [18de7e97]` has same name or UUID as the active project

In [3]:
# Load MyMethods.jl into your Julia session 
using MyMethods

┌ Info: Precompiling MyMethods [18de7e97-ce2a-4ea9-8d3b-e4b0c3b0b515]
└ @ Base loading.jl:1278
┌ Warning: Package MyMethods does not have DataFrames in its dependencies:
│ - If you have MyMethods checked out for development and have
│   added DataFrames as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with MyMethods
└ Loading DataFrames into MyMethods from project dependency, future warnings for MyMethods are suppressed.


### Exemplary Data  

Generate basic data to be used in the examples.

In [52]:
# Data parameters
N = 10000 # sample size
beta = [1,2,-1,4] # regression coefficients

# Data
_r = (1 .+ (rand(N) .- 0.5)) # running variable
X = hcat(ones(N), rand(N,3)) # regressor matrix
y = X*beta.*_r + randn(N,1); # outcome simulation

### myLS
myLS() generates a (weighted) least squares object.

In [53]:
# Estimate least squares fit
fit_LS = myLS(y, X);

In [54]:
# Calculate and print standard errors
inference(fit_LS);

,coef,se,t-stat,p-val
,Float64,Float64,Float64,Float64
1,0.995508,0.0469052,21.2238,5.75365e-100
2,1.97273,0.0514144,38.3691,4.35e-322
3,-0.979123,0.0511049,-19.1591,8.13039e-82
4,4.01888,0.051504,78.0304,0.0


### myLLR
myLLR generates a local linear regression object.

In [62]:
# Load the Statistics.jl package for the quantile function
using Statistics

In [57]:
# Estimate local constant regression with bandwidth h = 0.3
fit_LLR = myLLR(y, _r, K=0, h=0.3);

In [67]:
# Calculate the local coefficients for quantiles of the running variable
_pred = quantile(_r, collect(1:100)/100)
coef_LLR = coef(fit_LLR, _pred)

100×1 Array{Float64,2}:
 2.1464778293523152
 2.164863583340293
 2.1848164925693503
 2.202379250049568
 2.2194945287863916
 2.238529367976072
 2.259072702473877
 2.2803928477555386
 2.295254367021799
 2.31565182883676
 2.3360382007757208
 2.3545523029576536
 2.372231630200885
 ⋮
 4.733934007346418
 4.752010549611426
 4.769309334451029
 4.787513610864694
 4.805460062302218
 4.825972099618868
 4.844847103487214
 4.86413108259053
 4.881314070357735
 4.899541943986124
 4.913817803302474
 4.930473781294121

In [64]:
quantile(_r, collect(1:100)/100)

100-element Array{Float64,1}:
 0.5098119748422718
 0.519929663411745
 0.5312425862687982
 0.5410527223031306
 0.5504979673201891
 0.5608259751974274
 0.5719217889601622
 0.5833940490414572
 0.5912824049846109
 0.60179137554664
 0.6119241601322797
 0.6211802838818781
 0.6300185570668358
 ⋮
 1.3886801745326134
 1.3983570083768275
 1.4076047765090018
 1.4175131678452177
 1.4273532764073946
 1.4384898085554276
 1.4488057259979312
 1.459505604148257
 1.4693402368492066
 1.4804654851334347
 1.4893536747517746
 1.4998952233928997